<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 8 - Model Subclassing with Keras - Aladdin Persson anlattı.
</font>

**Video**: [TensorFlow Tutorial 8 - Model Subclassing with Keras](https://www.youtube.com/watch?v=WcZ_1IAH_nM&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=8)

### İçindekiler

**Loading Dataset**

**Preprocessing Dataset**

**Defining CNNBlock(layers.Layer) Class for Creating Convolutional Neural Network**

**Defining ResBlock(layers.Layer) and ResNet_Like(keras.Model) Class for Creating Neural Network Similar to Residual Neural Network (ResNet)**


### Kaynaklar

* Residual Neural Networkleri (ResNet) teorik olarak anlamak için DeepLearningAI kanalındaki Andrew NG'nin anlattığı [C4W2L03 Resnets](https://www.youtube.com/watch?v=ZILIbUvp5lk) videoyu izleyebilirsin.

### <font color="blue"> Giriş</font>

Bu notebookta bize flexibility sağlayan bir yöntem olarak model subclassing yapacağız. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### 1. Loading Dataset

In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 2. Preprocessing Dataset

In [5]:
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0  
x_test = x_test.reshape(-1, 28, 28,1).astype("float32") / 255.0

`x_train.reshape(-1, 28, 28, 1)` kodunda sondaki 1 channelı ifade ediyor. Channelı eklemek için reshape yaptık.

### 3. Defining CNNBlock Class for Creating Convolutional Neural Network

##### <font color="green">`class CNNBlock(layers.Layer)` ile **convolutional layerı**, **batch normalization** ve **relu** aktivasyonunu tek seferde çalıştıracağımız bir class oluşturuyoruz.  </font>

> CNN -> BatchNorm -> ReLU (Common structure)

Daha önce CNN'i çalıştırdığımızda önce BatchNorm yapmıştık sonrasında ReLU activation fonksiyonunu çalıştırmıştık. Bunu 10 kez yapmamız gerektiğini düşünelim çok fazla kod yazmamızı gerektirir bu durum. Bunu yapması için bir class oluşturabiliriz. 

In [6]:
class CNNBlock(layers.Layer): # layers.Layer'dan miras alıyoruz. backpropogation vs. yapılıyor burada.
  def __init__(self, out_channels, kernel_size=3):
    super(CNNBlock, self).__init__()
    self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
    self.bn = layers.BatchNormalization()
  
  def call(self, input_tensor, training=False): 
    x = self.conv(input_tensor)
    x = self.bn(x, training=training)
    x = tf.nn.relu(x)
    return x

`def call(self, input_tensor, training=False):` 
* call method pytorchtaki forward metod oluyor. Initte tanımladığımız layerları burada çağırıyoruz. Input tensoru alıyoruz ve bu layerlar üzerinden onları çalıştırıyoruz. 
* batchnormalization training ve evaluation modda farklı çalıştığı için çalıştırırken hangi modda olduğunu belirtmemiz gerekiyor True/False olarak. trainingin True/False olması olayı normalde model.fit ve model.evaluate içinde otomatik yapılıyor zaten model.fit'te False, model.evaluate'te True olarak.

Bu kodla defalarca çalıştırabiliriz bunu. 
> CNN -> BatchNorm -> ReLU (Common structure).

###### (Çalıştırma) 
<font color="purple">Her convolutional layerdan sonra inputumuzun shape'ini öğrenmek istiyorsak `print(x.shape)` gibi bir kod ekleyebiliriz classımıza. Bu şekilde debug yapabiliriz.  </font>

In [ ]:
class CNNBlock(layers.Layer): # layers.Layer'dan miras alıyoruz. backpropogation vs. yapılıyor burada.
  def __init__(self, out_channels, kernel_size=3):
    super(CNNBlock, self).__init__()
    self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
    self.bn = layers.BatchNormalization()
  
  def call(self, input_tensor, training=False): 
    x = self.conv(input_tensor)
    print(x.shape)
    x = self.bn(x, training=training)
    x = tf.nn.relu(x)
    return x

###### Devam

In [7]:
model = keras.Sequential(
    [
     CNNBlock(32), #out_channelsları yazıyoruz.
     CNNBlock(64),
     CNNBlock(128),
     layers.Flatten(),
     layers.Dense(10),
    ]
)

##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [8]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli eğitiyoruz. </font>

In [9]:
model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=2)

Epoch 1/3
938/938 - 53s - loss: 0.4763 - accuracy: 0.9497
Epoch 2/3
938/938 - 21s - loss: 0.0698 - accuracy: 0.9844
Epoch 3/3
938/938 - 21s - loss: 0.0332 - accuracy: 0.9891


In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cnn_block (CNNBlock)         (None, 28, 28, 32)        448       
_________________________________________________________________
cnn_block_1 (CNNBlock)       (None, 28, 28, 64)        18752     
_________________________________________________________________
cnn_block_2 (CNNBlock)       (None, 28, 28, 128)       74368     
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                1003530   
Total params: 1,097,098
Trainable params: 1,096,650
Non-trainable params: 448
_________________________________________________________________
None


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [12]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 2s - loss: 0.0398 - accuracy: 0.9881


[0.03978950157761574, 0.988099992275238]

### 4. Defining ResBlock(layers.Layer) and ResNet_Like(keras.Model) Class for Creating Neural Network Similar to Residual Neural Network (ResNet)

In [13]:
class CNNBlock(layers.Layer): # yukarıda yazdığımız class
  def __init__(self, out_channels, kernel_size=3):
    super(CNNBlock, self).__init__()
    self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
    self.bn = layers.BatchNormalization()
  
  def call(self, input_tensor, training=False): 
    x = self.conv(input_tensor)
    x = self.bn(x, training=training)
    x = tf.nn.relu(x)
    return x

In [14]:
class ResBlock(layers.Layer):
  def __init__(self, channels): # 'channels'ı liste halinde vereceğiz. 
    super(ResBlock, self).__init__()
    self.cnn1 = CNNBlock(channels[0]) # kernel size default olarak 3'tü. Her CNNBlock için CNN, batch norm ve relu çalışacak.
    self.cnn2 = CNNBlock(channels[1])
    self.cnn3 = CNNBlock(channels[2])
    self.pooling = layers.MaxPooling2D() # input size'ın height ve widthini yarıya düşürecek. 
    # We're going to use an identity mapping similar to resnets with these skip connections. 
    self.identity_mapping = layers.Conv2D(channels[1], 1, padding='same')
    
  def call(self, input_tensor, training=False):
    x = self.cnn1(input_tensor, training=training)
    x = self.cnn2(x, training=training)
    x = self.cnn3(
        x + self.identity_mapping(input_tensor), training=training,
    )
    return self.pooling(x)


* `self.identity_mapping`  identity mapping has the same number of channels. We're using same convolution so the height and the width won't change but channels might. 

* `x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,))` what we're doing here is that we're using these skip connections here: `self.identity_mapping(input_tensor)` but for this one `...(x + ` it has passed through this `x = self.cnn2(x, training=training)` and changing the number of channels that it was originally to the number of channels that we send in through "channels" list (channels[1]). 

class ResBlock(layers.Layer)'ı final model olarak kullanmayacağız. Final model için `keras.Model`'i inherit ettiğimiz bir class yazıyoruz. Bu parent, layerlara sahip ayrıca ek olarak built-in training, evaluation gibi `model.fit`ten, `model.evaluate`ten, `model.predict`ten bildiğimiz özelliklere hatta model.layers (modeldeki layerları kontrol etmek için), model.summary, serialization, saving your model gibi özelliklere de sahip. Bunlar `keras.Model`'de var ama `layers.Layer`'da yok. Bu yüzden bu classı oluşturuyoruz.

In [19]:
class ResNet_Like(keras.Model):
  def __init__(self, num_classes=10):
    super(ResNet_Like, self).__init__()
    self.block1 = ResBlock([32, 32, 64])
    self.block2 = ResBlock([128, 128, 256])
    self.block3 = ResBlock([128, 256, 512])
    self.pool = layers.GlobalAveragePooling2D() # bu self.Flatten ile aynı fonksiyonaliteyi gerçekleştiriyor. 
    self.classifier = layers.Dense(num_classes)

  def call(self, input_tensor, training=False): 
    x = self.block1(input_tensor, training=training)
    x = self.block2(x, training=training)
    x = self.block3(x, training=training)
    x = self.pool(x)
    return self.classifier(x)

  def model(self):
    x = keras.Input(shape=(28,28,1))
    return keras.Model(inputs=[x], outputs=self.call(x))


* İlk olarak CNNBlock'u oluşturduk sadece convolutional layerı, batch normalization ve reluyu defalarca kez elle yazmamak için.  

* İkinci olarak ResBlock'u yazdık. Bu class CNNBlockları kullanıyor birkaç kez ve pooling layer ve identity mapping kullanıyor. 

* Son olarak Resnet_Like classı yazıyoruz ve içerisinde ResBlocku  kullanıyoruz. Flatten işlemini burada yapıyoruz.

* `print(model.summary())` yazdığımızda Output Shape'ini "multiple" olarak veriyordu. `def model(self)` ile input shape'ini kendimiz verip output shape'ini dönen bir fonksiyon yazmış olduk.


###### Devam

In [21]:
model = ResNet_Like(num_classes=10)

##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [22]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli eğitiyoruz. </font>

In [23]:
model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2) # epochs=20 yaparsak trainde 0.9984 testte 0.9914 accuracylere ulaşabiliyoruz.

938/938 - 49s - loss: 0.0822 - accuracy: 0.9751


In [24]:
print(model.summary())

Model: "res_net__like_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
res_block_6 (ResBlock)       multiple                  28640     
_________________________________________________________________
res_block_7 (ResBlock)       multiple                  526976    
_________________________________________________________________
res_block_8 (ResBlock)       multiple                  1839744   
_________________________________________________________________
global_average_pooling2d_2 ( multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  5130      
Total params: 2,400,490
Trainable params: 2,397,418
Non-trainable params: 3,072
_________________________________________________________________
None


In [25]:
print(model.model().summary()) # class içinde tanımladığımız fonksiyonu çağırdık böylelikle.

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
res_block_6 (ResBlock)       (None, 14, 14, 64)        28640     
_________________________________________________________________
res_block_7 (ResBlock)       (None, 7, 7, 256)         526976    
_________________________________________________________________
res_block_8 (ResBlock)       (None, 3, 3, 512)         1839744   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 2,400,490
Trainable params: 2,397,418
Non-trainable params: 3,072
_______________________________________________

global_average_pooling2d kısmında 3x3'lük (3 piksel height 3 piksel width) inputun ortalamasını alarak tek inputa indirdi. Baştaki input size'ın her seferinde yarıya düşmesinin sebebi maxpoolingden kaynaklı.

Total params: 2,400,490 bu sayı önceki modellerimizden önemli oranda fazla. 

##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [26]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 3s - loss: 0.0277 - accuracy: 0.9912


[0.027706323191523552, 0.9911999702453613]